In [1]:
import numpy as np
import pandas as pd
from scipy.stats import pearsonr
from tqdm.notebook import trange
np.random.seed(1234)

In [2]:
class Buyer:
    # This class defines a buyer. A buyer has a reservation value above
    # which he will not trade. Since each buyer can only trade once, we
    # record whether a buyer traded or not. A buyer generates a bid price
    # (which varies depending if the simulation runs with or without
    # constraint)
    
    def __init__(self, quantity):
        self.Value = []
        self.Traded = 0
        self.Quantity = quantity
    
    # Generate a bid offer
    def formBidPrice(self, bound, threshold):
        Determiner = 0
        if threshold == 0:
            Determiner = 1
        else:
            Determiner = np.random.rand()
        if Determiner >= threshold:
            potentialBid = np.random.randint(0, self.Value[self.Traded]+1)
        else:
            print("yes")
            potentialBid = np.random.randint(0, bound+1)
            
        return potentialBid

In [3]:
class Seller:
    # This class defines a seller. A seller has a reservation cost below
    # which he will not trade. Since each seller can only trade once, we
    # record whether a seller traded or not. A seller generates a bid price
    # (which varies depending if the simulation runs with or without
    # constraint)
    
    def __init__(self, quantity):
        self.Cost = []
        self.Traded = 0
        self.Quantity = quantity
    
    # Generate ask offer
    def formAskPrice(self, bound, threshold):
        Determiner = 0
        if threshold == 0:
            Determiner = 1
        else:
            Determiner = np.random.rand()
            
        if Determiner >= threshold:
            potentialAsk = np.random.randint(self.Cost[self.Traded], bound+1)
        else:
            potentialAsk = np.random.randint(0, bound+1)
        
        return potentialAsk

In [4]:
def initializeBook(bound):
    # Initialize order book variables. Record the best standing bid and ask,
    # the ID of the best bidders (index number in buyer/seller vectors), 
    # transaction price (initialized to 0) and logical variables to determine
    # if there currently is a standing bid or ask. Initially there are no best 
    # bidders, so index of each is 0. The order book vector also records the
    # added value of a given trade (initialized to 0)

    # First index is the best bid (initialized to 0) 0
    # Second index is the best bid ID (initialized to 0, nobody) 1
    # Third index is the logical variable for current bid (1 for true, initialized to 0) 2
    # Fourth index is the best ask (initialized to the max cost for sellers) 3
    # Fifth index is the best ask ID (initialized to 0, nobody) 4
    # Sixth index is the logical variable for current ask (1 for true, initialized to 0). 5
    # Seventh index is the transaction price (initialized to 0) 6
    
    orderBookValues = [0 ,-1 , 0, bound, -1, 0, 0, 0] 
    return orderBookValues

In [5]:
def doTrade(buyers, sellers, bookValues, Quantity, bound, threshold):
    # This function operates the double auction order book. The function
    # determines with 50% probability whether the next trader to submit an
    # offer is a buyer or a seller. Then, it compares the new bid/ask and
    # executes a trade if it satisfies the counterposing best standing offer,
    # or it takes the place as best offer of its kind if it is better than the
    # currently standing one (for example, if a new bid is submitted and it is
    # higher than the best ask, a trade will occur, but if it's not higher than
    # the best ask but higher than the best standing bid it will take its
    # place. A similar process takes place if a seller submits an ask). The
    # function returns an updated state of the order book to update simulation.
    
    # Initialize return vector to current book values
    updatedValues = bookValues
    
    num_remaining_b = sum([b.Traded < b.Quantity for b in buyers])
    num_remaining_s = sum([s.Traded < s.Quantity for s in sellers])
    
    b_ratio = num_remaining_b / (num_remaining_b + num_remaining_s)
    

    # Randomly choose between a buyer and a seller (belor 0.5 we select a
    # buyer, otherwise we select a seller).
    traderDeterminer = np.random.rand()
    # Process as buyer
    if traderDeterminer < b_ratio:
        
        # Initialize a new buyer ID (at first 0)
        newBuyer = -1
    
        #Choose a random buyer that has yet not traded. Lock selection of a
        # buyer in a while loop until it chooses a buyer that hasn't traded
        while newBuyer == -1:
            # Choose a random index in the buyers vector to select a buyer
            randomIndex = np.random.randint(0, len(buyers))
            if (buyers[randomIndex].Traded != Quantity):
                newBuyer = randomIndex
        
        
        # Make buyer generate a random bid based on its reservation price and
        # simulation constraint
        newBid = buyers[newBuyer].formBidPrice(bound, threshold)
    
        # Do a trade if there is currently a standing ask that the bid can
        # trade with. Check the logical variable and the value of the best
        # standing ask
        if (updatedValues[5] == 1) & (newBid >= updatedValues[3]):
            # Set the transaction price to the best ask value
            updatedValues[6] = updatedValues[3]
            # Record surplus added by the trade
            # updatedValues[7] = buyers[newBuyer].Value - sellers[updatedValues[4]].Cost
            # Update ID of buyer
            updatedValues[1] = newBuyer
            updatedValues[7] = 1

        
        # If the there is no trade, set the bid as best bid if it is higher
        # than the currently standing best bid, even if it doesn't satisfy the
        # ask or if there currently is no ask
        else:
            if (newBid > updatedValues[0]) | (updatedValues[2] == 0):
                # Set new bid as best bid, and update the ID of bidder
                updatedValues[0]= newBid
                updatedValues[1] = newBuyer
                # Set logical variable for standing bid to true
                updatedValues[2] = 1
                # If new bid is lower than best bid, do nothing
    # Process a seller
    else:
        # Initialize a new seller ID (at first 0)
        newSeller = -1
        # Choose a random seller that has yet not traded. Lock selection of a
        # seller in a while loop until it chooses a seller that hasn't traded
        while newSeller == -1:
            # Choose a random index in the sellers vector to select a seller
            randomIndex = np.random.randint(0, len(sellers))
            if (sellers[randomIndex].Traded != Quantity):
                newSeller = randomIndex
        
        # Make seller generate a random ask based on its reservation cost and
        # simulation constraint
        
        newAsk = sellers[newSeller].formAskPrice(bound, threshold)
        
        # Do a trade if there is currently a standing bid that the ask can
        # trade with. Check the logical variable and the value of the best
        # standing bid
        if (updatedValues[2] == 1) & (updatedValues[0] >= newAsk):
            # Set the transaction price to the best bid value
            updatedValues[6] = updatedValues[0]
            # Record surplus added by the trade
            #updatedValues[7] = buyers[updatedValues[1]].Value - sellers[newSeller].Cost
            # Record ID of seller
            updatedValues[4] = newSeller
            updatedValues[7] = 1
            
        # If the there is no trade, set the ask as best ask if it is lower
        # than the currently standing best ask, even if it doesn't satisfy the
        # bid or if there currently is no bid
        else:
            if (newAsk < updatedValues[3]) | (updatedValues[5] == 0):
                # Set new ask as best ask, and update the ID of bidder
                updatedValues[3] = newAsk
                updatedValues[4] = newSeller
                # Set logical variable for standing ask to true
                updatedValues[5] = 1
                # If new ask is higher than best ask, do nothing
       
    return updatedValues

In [6]:
def Average(lst): 
    return sum(lst)/len(lst)

def Rank(vector):
    a={}
    rank=1
    for num in sorted(vector):
        if num not in a:
            a[num]=rank
            rank=rank+1
    return[a[i] for i in vector]

def Possible_trade(buyers, sellers, Quantity):
    current_v = []
    current_c = []
    for b in buyers:
        if b.Traded != Quantity:
            current_v.append(b.Value[b.Traded])
    for s in sellers:
        if s.Traded != Quantity:
            current_c.append(s.Cost[s.Traded])

    if (current_v and current_c):
        if max(current_v) < min(current_c):
            return False
    else:
        return False
    return True

def compute_EM_inefficiency(eq_price, eq_quantity,
                            optimalconsumersurplus, optimalproducersurplus, 
                            optimal_surplus, hist):
    EM = []
    for period in hist:
        Quantity = len(period)
        buyervalue = [trans[1] for trans in period]
        sellercost = [trans[0] for trans in period]
        price = [trans[2] for trans in period]
        
        ConsumerSurplus = sum([buyervalue[i] - price[i] for i in range(len(price))])
        ProducerSurplus = sum([price[i] - sellercost[i] for i in range(len(price))])
        efficiency = (ConsumerSurplus + ProducerSurplus) / optimal_surplus
        
        
        # EM-inefficiency
        buyer_surplus_from_trade = [bv - eq_price for bv in buyervalue]
        seller_surplus_from_trade = [eq_price - sc for sc in sellercost]
        
        # buyer side
        buyer_total_surplus_positive = sum([s for s in buyer_surplus_from_trade if s >= 0])
        buyer_surplus_diff = optimalconsumersurplus - buyer_total_surplus_positive
        
        buyer_total_surplus_negative = sum([-s for s in buyer_surplus_from_trade if s < 0])
        
        infra_buyer = len([s for s in buyer_surplus_from_trade if s >= 0])
        missing_infra_buyer = eq_quantity - infra_buyer
        extra_buyer = len(period) - infra_buyer
        
        # seller side
        seller_total_surplus_positive = sum([s for s in seller_surplus_from_trade if s >= 0])
        seller_surplus_diff = optimalproducersurplus - seller_total_surplus_positive
        
        seller_total_surplus_negative = sum([-s for s in seller_surplus_from_trade if s < 0])
        
        infra_seller = len([s for s in seller_surplus_from_trade if s >= 0])
        missing_infra_seller = eq_quantity - infra_seller
        extra_seller = len(period) - infra_seller
        
        EM_inefficiency = -2
        if (Quantity >= eq_quantity) & (efficiency != 1):
            EM_inefficiency = 1
        elif (Quantity < eq_quantity) & (extra_buyer == 0) & (extra_seller == 0):
            EM_inefficiency = 0
        elif efficiency == 1.0:
            EM_inefficiency = -1
        else:
            v_score_buyer = buyer_surplus_diff*(missing_infra_buyer - extra_buyer)/missing_infra_buyer
            em_score_buyer = buyer_surplus_diff - v_score_buyer + buyer_total_surplus_negative
            
            v_score_seller = seller_surplus_diff*(missing_infra_seller - extra_seller)/missing_infra_seller
            em_score_seller = seller_surplus_diff - v_score_seller + seller_total_surplus_negative
            
            em_score = em_score_buyer + em_score_seller
            v_score = v_score_seller + v_score_buyer
            em_proportion = em_score/(em_score + v_score)
            EM_inefficiency = em_proportion
        
        if EM_inefficiency != -1:
            EM.append(EM_inefficiency)
            
    sum_stat = {
        "indice": ["EM_inefficiency"],
        "avg": [Average(EM)],
        "sd": [np.std(EM)]
    }
    
    summary = pd.DataFrame(sum_stat)
            
    return summary



In [7]:
def simulation(setting, random_seed, p, times, iteration = 0):
    
    eq_price = 112
    eq_quantity = 13
    optimalconsumersurplus = 260
    optimalproducersurplus = 416
    optimal_surplus = 676

    numberBuyers = setting["numberBuyers"]
    numberSellers = setting["numberSellers"]
    Quantity = setting["Quantity"]
    bound = setting["bound"]
    # Set the threshold so that if a d following U(0, 1) < d, the trader use true random
    threshold = p

    np.random.seed(random_seed)
    
    Values = setting["Values"]
    Costs = setting["Costs"]
    
    # Create vector holding autocorreltion of price change
    # and correlation of transaction order and costs/values
    auto_corr = []
    trx_order_seller = []
    trx_order_buyer = []
    q = []
    p = []
    hist = []
    buyervalue = []
    sellercost = []
    price = []
    simulation = []
    all_books = []
    itr = []

    # Create vector holding all buyers
    buyers = []
    for i in range(numberBuyers):
        buyers.append(Buyer(Quantity))

    # Create vector holding all sellers
    sellers = []
    for i in range(numberSellers):
        sellers.append(Seller(Quantity))
        
    # Conduct simulation for 10,000 times
    for j in trange(times):
        # Initialize all buyers to not traded state and give them a reservation
        # price (random variable bounded above)
        valueVec = np.asarray(Values)

        for i in range(numberBuyers):
            valueVec[i].sort()
            valueVec[i] = valueVec[i][::-1]
            buyers[i].Value = valueVec[i]
            buyers[i].Traded = 0


        # Initialize all sellers to not traded state and give them a reservation
        # cost (random variable bounded below)
        costVec = np.asarray(Costs)

        for i in range(numberSellers):
            costVec[i].sort()
            sellers[i].Cost = costVec[i]
            sellers[i].Traded = 0


        # Initialize the order book vector. Please see function description for the
        # values held in each index   
        orderBookValues = initializeBook(bound)

        # Initialize vector with transaction prices, values, and costs (update as iterations execute).
        transactionPrices = []

        # Initial quantity traded is 0
        #quantity = 0
        tradedValues = []
        tradedCosts  = []
        h = []

        count = 0

        while True:

            count += 1
            
            if iteration > 0:
                if count > iteration:
                    break

            if not Possible_trade(buyers, sellers, Quantity):
                break
                

            # Attempt a trade or a new bid/ask (report update if trade occurs). 
            orderBookValues = doTrade(buyers, sellers, orderBookValues, Quantity, bound, threshold)
            
            # Record transaction price, update surplus and quantity, mark traders 
            # to record that they have traded, and reinitialize the order book
            # if a trade occured
            if orderBookValues[7] == 1:
                simulation.append(j+1)
                transactionPrices.append(orderBookValues[6])
                price.append(orderBookValues[6])
                tradedValues.append(buyers[orderBookValues[1]].Value[buyers[orderBookValues[1]].Traded])
                buyervalue.append(buyers[orderBookValues[1]].Value[buyers[orderBookValues[1]].Traded])
                tradedCosts.append(sellers[orderBookValues[4]].Cost[sellers[orderBookValues[4]].Traded])
                sellercost.append(sellers[orderBookValues[4]].Cost[sellers[orderBookValues[4]].Traded])
                h.append([sellers[orderBookValues[4]].Cost[sellers[orderBookValues[4]].Traded], 
                          buyers[orderBookValues[1]].Value[buyers[orderBookValues[1]].Traded], 
                          orderBookValues[6]])
                buyers[orderBookValues[1]].Traded += 1
                sellers[orderBookValues[4]].Traded += 1
                orderBookValues = initializeBook(bound)    
    

        # gen transaction order and price change
        order = []
        for i in range(1, len(transactionPrices) + 1):
            order.append(i)

        all_priceChange = []
        for i in range(1, len(transactionPrices)):
            all_priceChange.append(transactionPrices[i] - transactionPrices[i - 1])

        priceChange = []
        lag_priceChange = []
        for i in range(1, len(all_priceChange)):
            priceChange.append(all_priceChange[i])
            lag_priceChange.append(all_priceChange[i-1])


        # Calculate correlation
        price_auto_corr, _ = pearsonr(priceChange, lag_priceChange)
        seller_corr, _ = pearsonr(Rank(tradedCosts), order)
        buyer_corr, _ = pearsonr(Rank(tradedValues), order)

        auto_corr.append(price_auto_corr)
        trx_order_seller.append(seller_corr)
        trx_order_buyer.append(buyer_corr)
        q.append(len(order))
        
        hist.append(h)
        itr.append(count)
    
    sum_stat = {
        "indice": ["trx_order_buyer", "trx_order_seller", "auto corr"],
        "avg": [Average(trx_order_buyer), Average(trx_order_seller), Average(auto_corr)],
        "sd": [np.std(trx_order_buyer), np.std(trx_order_seller), np.std(auto_corr)]
    }
    
    summary = pd.DataFrame(sum_stat)
    
    outcome = {
        "trx_order_buyer": trx_order_buyer,
        "trx_order_seller": trx_order_seller,
        "auto_corr": auto_corr,
        "q": q
    }
    output = pd.DataFrame(outcome)
    
    outcome_detail = {
        "buyervalue": buyervalue,
        "sellercost": sellercost,
        "price": price,
        "simulation": simulation
    }
    output_detail = pd.DataFrame(outcome_detail)
    results = {
        "summary": summary,
        "output": output,
        "output_detail": output_detail,
        "avg_q": Average(q),
        "hist": hist,
        "itr": itr
    }
    
    return results